In [9]:
import pandas as pd
import numpy as np

In [10]:
train_df = pd.read_csv("../stand_Data/train_stand.csv")
val_df = pd.read_csv("../stand_Data/val_stand.csv")
test_df = pd.read_csv("../stand_Data/test_stand.csv")

train_x = train_df[:]
val_x = val_df.drop('Class',axis=1)
val_y = val_df['Class']

In [11]:
#functions
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [12]:
from sklearn.ensemble import IsolationForest 
from sklearn.metrics import classification_report,f1_score

val_contamination = val_y.mean()
print(val_contamination)


0.0010540369615627855


In [13]:
colnames = ['V12']

In [14]:

model = IsolationForest(n_estimators=125, max_samples=len(train_x), contamination=val_contamination, random_state=42, verbose=0)
model.fit(train_x.loc[:,colnames])

/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


IsolationForest(contamination=0.0010540369615627855, max_samples=113842,
                n_estimators=125, random_state=42)

In [15]:
from numpy import average


pred = model.predict(val_x.loc[:,colnames])
pred = get_pred_label(pred)

print(classification_report(pred,val_y))
print(f1_score(val_y, pred, average='macro'))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28427
           1       0.57      0.49      0.52        35

    accuracy                           1.00     28462
   macro avg       0.78      0.74      0.76     28462
weighted avg       1.00      1.00      1.00     28462

0.7612658573772909


In [17]:
#제출용
test_df = pd.read_csv('../Data/test.csv') # Train
test_df.head()
test_x = test_df.loc[:,colnames]
test_pred = model.predict(test_x) # model prediction
test_pred = get_pred_label(test_pred)
## Submission
submit = pd.read_csv('../Data/sample_submission.csv')
submit.head()
submit['Class'] = test_pred
submit.to_csv('./submit.csv', index=False)